In [1]:
# import pandas library
import pandas as pd

In [2]:
# read four csvs into four dataframes
df_tina=pd.read_csv('tina.csv')
df_yvette=pd.read_csv('yvette.csv')
df_thomas=pd.read_csv('final_walkscore_thomas.csv')
df_jamie=pd.read_csv('jamie.csv')

In [3]:
# take a brief look at each dataframe
df_tina.head()

,city,state,state_abbr,lat,lng,zipcode,aqi_pm2.5,park_score,water_violation_percent
0,New York,New York,NY,40.730646,-73.986614,10004,62.0,74.8,32.0
1,Los Angeles,California,CA,34.143916,-118.761144,90004,26.0,42.5,1.0
2,Chicago,Illinois,IL,41.875555,-87.624421,60604,42.0,77.2,6.0
3,Houston,Texas,TX,29.758938,-95.367697,77005,56.0,41.2,54.0
4,Phoenix,Arizona,AZ,33.448587,-112.077346,85007,28.0,39.6,17.0


In [4]:
df_yvette.head()

,city,cost_of_living_score
0,New York,100.00
1,Los Angeles,79.99
2,Chicago,77.34
3,Houston,63.09
4,Phoenix,69.41


In [5]:
df_jamie.head()

,city,state,state_abbr,edu_score,job_score,safety_score
0,New York,New York,NY,64.69,43.70,72.96
1,Los Angeles,California,CA,50.25,42.62,64.64
2,Chicago,Illinois,IL,61.17,45.15,71.43
3,Houston,Texas,TX,50.24,51.41,67.52
4,Phoenix,Arizona,AZ,52.27,53.48,71.89


In [6]:
# rename city_name and state_name columns for merge purpose (so have common key names)
df_thomas.rename(columns={'city_name':'city','state_name':'state'},inplace = True)
df_thomas.head()

,city,state,state_abbr,walk_score,bike_score
0,New York,New York,NY,100.0,97.0
1,Los Angeles,California,CA,55.0,51.0
2,Chicago,Illinois,IL,94.0,85.0
3,Houston,Texas,TX,84.0,64.0
4,Phoenix,Arizona,AZ,90.0,58.0


In [7]:
# merge all four dataframes
df=df_tina.merge(df_yvette, on='city',how='left').merge(
    df_thomas, on=['city','state'],how='left').merge(
    df_jamie, on=['city','state'], how='left')
df.head()

,city,state,state_abbr_x,lat,lng,zipcode,aqi_pm2.5,park_score,water_violation_percent,cost_of_living_score,state_abbr_y,walk_score,bike_score,state_abbr,edu_score,job_score,safety_score
0,New York,New York,NY,40.730646,-73.986614,10004,62.0,74.8,32.0,100.00,NY,100.0,97.0,NY,64.69,43.70,72.96
1,Los Angeles,California,CA,34.143916,-118.761144,90004,26.0,42.5,1.0,79.99,CA,55.0,51.0,CA,50.25,42.62,64.64
2,Chicago,Illinois,IL,41.875555,-87.624421,60604,42.0,77.2,6.0,77.34,IL,94.0,85.0,IL,61.17,45.15,71.43
3,Houston,Texas,TX,29.758938,-95.367697,77005,56.0,41.2,54.0,63.09,TX,84.0,64.0,TX,50.24,51.41,67.52
4,Phoenix,Arizona,AZ,33.448587,-112.077346,85007,28.0,39.6,17.0,69.41,AZ,90.0,58.0,AZ,52.27,53.48,71.89


In [8]:
# drop repeated state abbreviation columns
df=df.drop(columns=['state_abbr_x','state_abbr_y'])

In [9]:
# take a look at number of rows with null values
len(df.dropna())

51

In [10]:
# drop rows with null values
df=df.dropna()

In [11]:
# create a new dataframe by copying the orignal one for data normalization 
df_nor = df.copy()
for col in df_nor.columns:
    if col not in ['city','state','lat','lng','zipcode','state_abbr']:
        # normalize score related data using max-min normalization method
        df_nor[col] = df_nor[col]  / df_nor[col].abs().max() * 100
        # for variables in the opposite direction (higher the score, worse the situation)
        # deducted by 100 to reverse the direction
        if col in ['aqi_pm2.5','water_violation_percent','cost_of_living_score']:
            df_nor[col] = df_nor[col].apply(lambda x: 100-x)
            
# take a look at the normalized data
df_nor.head()

,city,state,lat,lng,zipcode,aqi_pm2.5,park_score,water_violation_percent,cost_of_living_score,walk_score,bike_score,state_abbr,edu_score,job_score,safety_score
0,New York,New York,40.730646,-73.986614,10004,7.462687,88.625592,63.636364,0.00,100.0,97.979798,NY,78.317191,70.438427,87.629114
1,Los Angeles,California,34.143916,-118.761144,90004,61.194030,50.355450,98.863636,20.01,55.0,51.515152,CA,60.835351,68.697614,77.636320
2,Chicago,Illinois,41.875555,-87.624421,60604,37.313433,91.469194,93.181818,22.66,94.0,85.858586,IL,74.055690,72.775629,85.791497
3,Houston,Texas,29.758938,-95.367697,77005,16.417910,48.815166,38.636364,36.91,84.0,64.646465,TX,60.823245,82.865893,81.095364
4,Phoenix,Arizona,33.448587,-112.077346,85007,58.208955,46.919431,80.681818,30.59,90.0,58.585859,AZ,63.280872,86.202450,86.343983


In [12]:
# calculate aggregated environment score and leisure & accessibility score

# take average of air quality index of PM2.5 and water violation percent as environment score
df_nor['environment_score'] = df[['aqi_pm2.5','water_violation_percent']].mean(axis=1)

# take average of park score, walk score, bike score as leisure & accessibility score
df_nor['leisure+accessibility_score'] = df[['park_score','walk_score','bike_score']].mean(axis=1)

In [13]:
# create the final dataframe by subsetting needed columns from the normalization dataframe
df_final = df_nor[['city','state','state_abbr','zipcode','lat','lng','environment_score','leisure+accessibility_score',
                  'cost_of_living_score','edu_score','job_score','safety_score']]
df_final['city'] = df_final['city'] + ', ' + df_final['state_abbr']
del df_final['state']
del df_final['state_abbr']
# take a look at the final dataframe
df_final.head()

d:\Users\tinaf\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,city,zipcode,lat,lng,environment_score,leisure+accessibility_score,cost_of_living_score,edu_score,job_score,safety_score
0,"New York, NY",10004,40.730646,-73.986614,47.0,90.600000,0.00,78.317191,70.438427,87.629114
1,"Los Angeles, CA",90004,34.143916,-118.761144,13.5,49.500000,20.01,60.835351,68.697614,77.636320
2,"Chicago, IL",60604,41.875555,-87.624421,24.0,85.400000,22.66,74.055690,72.775629,85.791497
3,"Houston, TX",77005,29.758938,-95.367697,55.0,63.066667,36.91,60.823245,82.865893,81.095364
4,"Phoenix, AZ",85007,33.448587,-112.077346,22.5,62.533333,30.59,63.280872,86.202450,86.343983


In [14]:
# final dataframe output as csv 
df_final.to_csv('merged_final.csv',index=False)